# Imports

Necessários para rodar no colab

In [ ]:
# Necessário pra rodar no Colab
# !pip uninstall accelerate transformers

In [ ]:
# Esse também 
# !pip install accelerate transformers

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import accelerate

/home/tales/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
# Tokenizador e modelo do BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

/home/tales/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Dataset de treino
train = pd.read_csv('train_enhanced.csv')
train.head(5)

,text,tokenized_text,homophobia,obscene,insult,racism,sexism,xenophobia
0,mano segundo billboard forbes bts precisou ads...,"['mano', 'segundo', 'Billboard', 'forbe', 'bts...",0,0,0,0,0,0
1,wanda tom pqp casalzão dois,"['wanda', 'tom', 'pqp', 'casalzãor', 'dois']",0,1,0,0,0,0
2,acabou sair operação assim kkkkkkkkkkkk,"['acabar', 'sair', 'operação', 'assim', 'kkkkk...",0,0,0,0,0,0
3,antes bruges filosofia turismo redenção,"['antes', 'bruges', 'filosofia', 'turismo', 'r...",0,0,0,0,0,0
4,porra vão abraçar mara bicho desde ontem ressu...,"['porra', 'ir', 'abraçar', 'mar', 'bicho', 'de...",0,0,0,0,0,0


In [12]:
# Dataset de teste
test = pd.read_csv('test_enhanced.csv')
test.head(5)

,text,tokenized_text,homophobia,obscene,insult,racism,sexism,xenophobia
0,lazarento skskkskskskskskskksksksksksksksksksk...,"['lazarento', 'skskkskskskskskskksksksksksksks...",0,0,1,0,0,0
1,mamando mlk maconheiro rola bonita macia trans...,"['mamar', 'mlk', 'maconheiro', 'rola', 'bonito...",0,1,0,0,0,0
2,sempre pinto cabelo mãe pinto mão inteira raiva,"['sempre', 'pinto', 'Cabelo', 'mãe', 'Pinto', ...",0,0,0,0,0,0
3,famosa morde assopra canalha mulherzinha outra...,"['famoso', 'morde', 'assopro', 'canalha', 'mul...",0,0,1,0,0,0
4,bichinha fôlego tadinha,"['bichinho', 'fôlego', 'tadinho']",0,0,0,0,0,0


# Preparo dos dados

In [5]:
class CustomDataset(Dataset):
    def __init__(self, tensor_dataset):
        self.tensor_dataset = tensor_dataset

    def __getitem__(self, idx):
        input_ids, attention_mask, labels = self.tensor_dataset[idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

    def __len__(self):
        return len(self.tensor_dataset)

## Dataset de Treino

In [6]:
# Tokenização e conversão pra tensores
inputs = tokenizer(train['tokenized_text'].tolist(), truncation=True, padding=True, max_length=128)
input_ids = torch.tensor(inputs['input_ids'])
attention_mask = torch.tensor(inputs['attention_mask'])

In [7]:
# Preparo dos rótulos
labels = train[['homophobia', 'obscene', 'insult', 'racism', 'sexism', 'xenophobia']].values
labels = torch.tensor(labels, dtype=torch.float)

In [8]:
# Criação do dataset
train_dataset = TensorDataset(input_ids, attention_mask, labels)

In [9]:
# Convertendo TensorDataset para CustomDataset
train_dataset = CustomDataset(train_dataset)

In [10]:
print(train_dataset[0])

{'input_ids': tensor([  101,  1031,  1005,  2158,  2080,  1005,  1010,  1005,  7367, 12734,
         3527,  1005,  1010,  1005,  4908,  1005,  1010,  1005,  2005,  4783,
         1005,  1010,  1005, 18411,  2015,  1005,  1010,  1005,  3653, 22987,
         2906,  1005,  1010,  1005, 14997,  1005,  1010,  1005, 10975,  2050,
         1005,  1010,  1005, 28774,  1005,  1010,  1005,  5328,  1005,  1010,
         1005,  1047, 19658, 19658, 19658,  1005,  1010,  1005, 18204,  3672,
         2063,  1005,  1010,  1005,  8183,  6843,  1005,  1010,  1005,  8473,
         2378,  3270,  1005,  1010,  1005,  6643,  6843,  1005,  1010,  1005,
        14997,  1005,  1010,  1005,  6583,  2080,  1005,  1010,  1005,  7110,
         2850,  1005,  1010,  1005,  6904,  8017,  1005,  1010,  1005, 10975,
        12070,  2050,  1005,  1010,  1005, 13675,  2401,  2099,  1005,  1010,
         1005,  4241, 17258,  2050,  1005,  1010,  1005, 17540,  2890,  1005,
         1010,  1005,  2613,  1005,  1010,  1005, 

## Dataset de Validação

In [13]:
# Tokenização e conversão pra tensores
inputs = tokenizer(test['tokenized_text'].tolist(), truncation=True, padding=True, max_length=128)
input_ids = torch.tensor(inputs['input_ids'])
attention_mask = torch.tensor(inputs['attention_mask'])

In [14]:
# Preparo dos rótulos
labels = test[['homophobia', 'obscene', 'insult', 'racism', 'sexism', 'xenophobia']].values
labels = torch.tensor(labels, dtype=torch.float)

In [15]:
# Criação do dataset
test_dataset = TensorDataset(input_ids, attention_mask, labels)

In [16]:
test_dataset = CustomDataset(test_dataset)

In [16]:
print(test_dataset[0])

{'input_ids': tensor([  101,   164,   112,  1084,  4803,   265, 22280,   112,   117,   112,
          139,  3472, 22331,  3472,  3472,  3472,  3472,  3472,  3472, 22331,
         3472,  3472,  3472,  3472,  3472,  3472,  3472,  3472,  3472,  1949,
        22331,  1949,   112,   166,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

# Treinamento do Modelo

In [17]:
# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results5',          # Diretório de saída para os resultados do modelo
    num_train_epochs=5,              # Número total de épocas de treinamento
    per_device_train_batch_size=64,   # Tamanho do lote de treinamento por dispositivo
    per_device_eval_batch_size=128,   # Tamanho do lote de avaliação por dispositivo
    warmup_steps=330,                 # Número de etapas de aquecimento para o otimizador
    weight_decay=0.1,                 # Decaimento de peso para aplicar
    logging_dir='./logs5',            # Diretório de saída para os logs de treinamento
    save_steps=330,
    logging_steps=330,
    eval_steps=330
)

In [18]:
# Inicializar o Trainer
trainer = Trainer(
    model=model,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)

In [19]:
# Check the type of the dataset
print("Type of test_dataset:", type(test_dataset))

# Check the length of the dataset
print("Length of test_dataset:", len(test_dataset))

# Check the type and value of the first item in the dataset
first_item = test_dataset[0]
print("Type of first item in test_dataset:", type(first_item))
print("First item in test_dataset:", first_item)

Type of test_dataset: <class '__main__.CustomDataset'>
Length of test_dataset: 7034
Type of first item in test_dataset: <class 'dict'>
First item in test_dataset: {'input_ids': tensor([  101,  1031,  1005,  2474,  9057,  4765,  2080,  1005,  1010,  1005,
        15315,  6711,  5705,  5705,  5705,  5705,  5705,  5705, 19658,  6711,
         6711,  6711,  6711,  6711,  6711,  6711,  6711,  8337, 15714,  1005,
         1033,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0, 

In [20]:
trainer.train()

 20%|██        | 330/1650 [04:12<15:54,  1.38it/s]

{'loss': 0.3029, 'grad_norm': 0.6381723880767822, 'learning_rate': 5e-05, 'epoch': 1.0}


 40%|████      | 660/1650 [08:34<11:51,  1.39it/s]

{'loss': 0.2053, 'grad_norm': 0.7551156282424927, 'learning_rate': 3.7500000000000003e-05, 'epoch': 2.0}


 60%|██████    | 990/1650 [12:55<07:55,  1.39it/s]

...

{'loss': 0.1519, 'grad_norm': 0.8226894736289978, 'learning_rate': 1.25e-05, 'epoch': 4.0}


100%|██████████| 1650/1650 [21:38<00:00,  1.38it/s]

{'loss': 0.128, 'grad_norm': 1.2104814052581787, 'learning_rate': 0.0, 'epoch': 5.0}


100%|██████████| 1650/1650 [21:39<00:00,  1.27it/s]


{'train_runtime': 1299.9533, 'train_samples_per_second': 81.161, 'train_steps_per_second': 1.269, 'train_loss': 0.192913862286192, 'epoch': 5.0}


In [21]:
checkpoint_330 = './results5/checkpoint-330'
model_eval = BertForSequenceClassification.from_pretrained(checkpoint_330)
# Inicializar o Trainer
trainer_eval = Trainer(
    model=model_eval,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)
trainer_eval.evaluate()

100%|██████████| 55/55 [00:28<00:00,  1.92it/s]


{'eval_loss': 0.22553236782550812,
 'eval_runtime': 28.819,
 'eval_samples_per_second': 244.075,
 'eval_steps_per_second': 1.908}

In [22]:
checkpoint_660 = './results5/checkpoint-660'
model_eval = BertForSequenceClassification.from_pretrained(checkpoint_660)
# Inicializar o Trainer
trainer_eval = Trainer(
    model=model_eval,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)
trainer_eval.evaluate()

100%|██████████| 55/55 [00:29<00:00,  1.86it/s]


{'eval_loss': 0.19899725914001465,
 'eval_runtime': 29.635,
 'eval_samples_per_second': 237.355,
 'eval_steps_per_second': 1.856}

In [23]:
checkpoint_990 = './results5/checkpoint-990'
model_eval = BertForSequenceClassification.from_pretrained(checkpoint_990)
# Inicializar o Trainer
trainer_eval = Trainer(
    model=model_eval,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)
trainer_eval.evaluate()

100%|██████████| 55/55 [00:29<00:00,  1.86it/s]


{'eval_loss': 0.19268633425235748,
 'eval_runtime': 29.6418,
 'eval_samples_per_second': 237.3,
 'eval_steps_per_second': 1.855}

In [26]:
checkpoint_1320 = './results5/checkpoint-1320'
model_eval = BertForSequenceClassification.from_pretrained(checkpoint_1320)
# Inicializar o Trainer
trainer_eval = Trainer(
    model=model_eval,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)
trainer_eval.evaluate()

100%|██████████| 55/55 [00:27<00:00,  1.97it/s]


{'eval_loss': 0.19645288586616516,
 'eval_runtime': 27.9856,
 'eval_samples_per_second': 251.343,
 'eval_steps_per_second': 1.965}

In [27]:
checkpoint_1650 = './results5/checkpoint-1650'
model_eval = BertForSequenceClassification.from_pretrained(checkpoint_1650)
# Inicializar o Trainer
trainer_eval = Trainer(
    model=model_eval,                         # O modelo a ser treinado
    args=training_args,                  # Argumentos de treinamento
    train_dataset=train_dataset,         # Conjunto de dados de treinamento
    eval_dataset=test_dataset,
)
trainer_eval.evaluate()

100%|██████████| 55/55 [00:29<00:00,  1.89it/s]


{'eval_loss': 0.20299679040908813,
 'eval_runtime': 29.1084,
 'eval_samples_per_second': 241.649,
 'eval_steps_per_second': 1.889}